# Industries

In [3]:
import pandas as pd
from sqlalchemy import create_engine

## Explore siccodes csv

In [2]:
siccd_codes = pd.read_csv('./siccodes12.csv')
siccd_codes.head()

,Unnamed: 0,start,end,industry
0,0,100,999,Consumer Nondurables
1,1,2000,2399,Consumer Nondurables
2,2,2700,2749,Consumer Nondurables
3,3,2770,2799,Consumer Nondurables
4,4,3100,3199,Consumer Nondurables


## Connect to database

In [4]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

## GET DATA

In [33]:
df = pd.read_sql(
    """
    select date, ticker, bm, roeq, ret, siccd
    from data
    where date='2020-01'
    order by date, ticker
    """,
    conn
  )
df = df.dropna()
df = df.set_index(["date", "ticker"])

In [34]:
df.head()

bm      roeq       ret  siccd
date    ticker                                     
2020-01 A       0.221863  0.037268 -0.032235   3826
        AAL    -0.011426 -1.040881 -0.064156   4512
        AAMC   -4.432845 -0.013082  0.093927   6211
        AAME    2.085421 -0.039700  0.121827   6320
        AAN     0.623240  0.023518  0.039398   7359

## Get Industry Example

In [28]:
def get_industry(sicc):
    filtered_row = siccd_codes[((siccd_codes['start'] < sicc) & (sicc < siccd_codes['end']))]
    return filtered_row['industry'].values[0] if not filtered_row.empty else 'Other'

## Add industry to a dataframe

In [35]:
industries = [get_industry(sicc) for sicc in df['siccd'].values.tolist()]
df['industry'] = industries

In [36]:
df.head()

bm      roeq       ret  siccd            industry
date    ticker                                                         
2020-01 A       0.221863  0.037268 -0.032235   3826  Business Equipment
        AAL    -0.011426 -1.040881 -0.064156   4512               Other
        AAMC   -4.432845 -0.013082  0.093927   6211             Finance
        AAME    2.085421 -0.039700  0.121827   6320             Finance
        AAN     0.623240  0.023518  0.039398   7359               Other

## Get dummies

In [37]:
from sklearn.linear_model import LinearRegression

d = pd.get_dummies(df.industry)
ind_names = d.columns.to_list()
features = ["bm", "roeq"] + ind_names
df2 = df.join(d)

model = LinearRegression(fit_intercept=False)
model.fit(df2[features], df2["ret"])
pd.Series(model.coef_, index=features)

bm                     -0.006009
roeq                   -0.007469
Business Equipment      0.004973
Chemicals              -0.068784
Consumer Durables      -0.002736
Consumer Nondurables   -0.037517
Energy                 -0.163522
Finance                -0.032537
Healthcare              0.025760
Manufacturing          -0.037312
Other                   0.008585
Shops                  -0.042680
Telecommunications      0.020978
Utilities               0.027846
dtype: float64

## One hot encoder

In [38]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline

transform = make_column_transformer(
    (OneHotEncoder(), ["industry"]),
    remainder="passthrough"
)

model = LinearRegression(fit_intercept=False)
pipe = make_pipeline(transform, model)
pipe.fit(df[["bm", "roeq", "industry"]], df["ret"])
model.coef_

array([ 0.00497305, -0.06878386, -0.00273576, -0.03751717, -0.16352194,
       -0.03253673,  0.02575975, -0.03731164,  0.00858517, -0.04267995,
        0.02097824,  0.02784589, -0.0060085 , -0.00746868])